In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from scipy.stats import norm
import pylab
tf.compat.v1.disable_eager_execution()

/Users/shankar/opt/miniconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/shankar/opt/miniconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/shankar/opt/miniconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/shanka

In [2]:
dimension_state = 1

riskfree_rate = 0.05
volatility = 0.25
strike_price = 0.5

# Time limits
T_min = 0.
T_max  = 1.

# Space limits
S_min = 1e-10 
S_max = 1.

# Network parameters
nr_layers = 3
nr_nodes_per_layer = 50
initial_learning_rate = 0.001
learning_rate_decay_steps = 10000
learning_rate_decay_rate = 0.9

# Training parameters
steps_per_sample = 1
nr_epochs = 8000

# Number of samples
N_interior = 1000
N_initial = 100

In [3]:
# DGM neural network model
class DNN(tf.keras.Model):#creating a class called DNN
    def __init__(self, nr_layers, nr_nodes_each_layer, state_dimension=1):#init is similiar to a constructor in c++
        #self allows you to call instances of that class similar to a this pointer.
        tf.keras.Model.__init__(self)#calls the parent constructor
        self.nr_layers = nr_layers #assinging member variables nr_layers as input varaible 

        self.initial_layer = DenseLayer(state_dimension + 1, nr_nodes_each_layer, activation=tf.nn.tanh)
        #setting parameters for initial layers start from 2(state dimension +time(1)) nodes all the way to 50 nodes
        self.hidden_layers = []
        for _ in range(nr_layers): #iterating over the 3 layers
            self.hidden_layers.append(LayerFromPaper(state_dimension + 1, nr_nodes_each_layer, activation=tf.nn.tanh))#appending the hidden layers create 3 of them
        self.final_layer = DenseLayer(nr_nodes_each_layer, 1, activation=None)#create the last layer


    def call(self, t, x):# creating of a member function
        X = tf.concat([t,x], 1) # concats the time and stock price in columns

        S = self.initial_layer.call(X)#call is a member function of dense layer
        for i in range(self.nr_layers):
            S = self.hidden_layers[i].call({'S': S, 'X': X})#creating the hidden layers, #X=time and asset price we concat this in  X = tf.concat([t,x], 1)
        result = self.final_layer.call(S)#creating the final layer

        return result
    


# Neural network layers

class DenseLayer(tf.keras.layers.Layer):# creating the class Dense layers
    def __init__(self, nr_inputs, nr_outputs, activation): #creating the constructor for that class
        tf.keras.layers.Layer.__init__(self)#initialzing the object of that class
        
        self.initializer = tf.keras.initializers.glorot_normal
        #self.initializer=tf.contrib.layers.xavier_initializer()) #TF 1

        self.nr_inputs = nr_inputs# initilaizing nr_inputs as a member variable
        self.nr_outputs = nr_outputs # initilizing nr_outputs as a member varaible
        
        self.W = self.add_variable("W", shape=[self.nr_inputs, self.nr_outputs],
                                   initializer=self.initializer())# initializing W as a member variable creating a matrix type object in order to train it for the neural network
        #W is one of the weights
        self.b = self.add_variable("b", shape=[1, self.nr_outputs])  #bias or constant added only at the end of training as therefore has no initilization

        self.activation = activation #saving it as a member variable
    
    
    def call(self, inputs):#member function of Dense Layer
        S = tf.add(tf.matmul(inputs, self.W), self.b) #From paper
        if not self.activation == None:
            S = self.activation(S) #activation function with the sigma (not volatility)

        return S



class LayerFromPaper(tf.keras.layers.Layer):
    def __init__(self, nr_inputs, nr_outputs, activation):
        tf.keras.layers.Layer.__init__(self)

        self.initializer = tf.keras.initializers.glorot_normal
        #self.initializer=tf.contrib.layers.xavier_initializer()) #TF 1
        
        self.nr_outputs = nr_outputs
        self.nr_inputs = nr_inputs

        self.Uz = self.add_variable("Uz", shape=[self.nr_inputs, self.nr_outputs],
                                    initializer=self.initializer())
        self.Ug = self.add_variable("Ug", shape=[self.nr_inputs, self.nr_outputs],
                                    initializer=self.initializer())
        self.Ur = self.add_variable("Ur", shape=[self.nr_inputs, self.nr_outputs],
                                    initializer=self.initializer())
        self.Uh = self.add_variable("Uh", shape=[self.nr_inputs, self.nr_outputs],
                                    initializer=self.initializer())
        self.Wz = self.add_variable("Wz", shape=[self.nr_outputs, self.nr_outputs],
                                    initializer=self.initializer())
        self.Wg = self.add_variable("Wg", shape=[self.nr_outputs, self.nr_outputs],
                                    initializer=self.initializer())
        self.Wr = self.add_variable("Wr", shape=[self.nr_outputs, self.nr_outputs],
                                    initializer=self.initializer())
        self.Wh = self.add_variable("Wh", shape=[self.nr_outputs, self.nr_outputs],
                                    initializer=self.initializer())
        self.bz = self.add_variable("bz", shape=[1, self.nr_outputs])
        self.bg = self.add_variable("bg", shape=[1, self.nr_outputs])
        self.br = self.add_variable("br", shape=[1, self.nr_outputs])
        self.bh = self.add_variable("bh", shape=[1, self.nr_outputs])

        self.activation = activation

    
    def call(self, inputs):
        S = inputs['S']
        X = inputs['X']

        Z = self.activation(tf.add(tf.add(tf.matmul(X, self.Uz), tf.matmul(S, self.Wz)), self.bz))
        G = self.activation(tf.add(tf.add(tf.matmul(X, self.Ug), tf.matmul(S, self.Wg)), self.bg))
        R = self.activation(tf.add(tf.add(tf.matmul(X, self.Ur), tf.matmul(S, self.Wr)), self.br))
        H = self.activation(tf.add(tf.add(tf.matmul(X, self.Uh), tf.matmul(tf.multiply(S, R), self.Wh)), self.bh))
        Snew = tf.add(tf.multiply(tf.subtract(tf.ones_like(G), G), H), tf.multiply(Z, S))

        return Snew

In [4]:
# Loss function
def get_residual(model, t_interior, x_interior, t_initial, x_initial):#model =BS Model,t_interior=time from 0 to 1,
    #x_interior=stock price from min to max, t_initial =t_min, x_initial=S_min
    # Loss term #1: PDE
    V = model(t_interior, x_interior)# valuation of the neural network
    V_t = tf.gradients(V, t_interior)[0] # DV/DT
    V_x = tf.gradients(V, x_interior)[0] # DV/DS
    V_xx = tf.gradients(V_x, x_interior)[0] #D^2V/DS^2
    #black scholes formula
    f = -V_t + riskfree_rate * V -riskfree_rate*x_interior*V_x - 0.5*volatility**2 * x_interior**2 * V_xx

    L1 = tf.reduce_mean(tf.square(f)) #mean of the squared residuals, residuals of the PDE J(F) part 1
    #Payoff function
    payoff= tf.math.maximum(0.,tf.subtract(x_interior,strike_price))
    #L2 norm
    L2=tf.reduce_mean(tf.square(tf.math.maximum(0.,payoff-V)))#J(f) part 2
    #max deviation of L2
    Max_dev=tf.math.reduce_max(tf.math.maximum(0.,payoff-V))
    #min deviation of L2
    Min_dev=tf.math.reduce_min(tf.math.maximum(0.,payoff-V))
    # Loss term #3: initial/terminal condition
    L3 = tf.reduce_mean(tf.square(model(t_initial,x_initial) - tf.math.maximum(0., x_initial - strike_price))) # J(F) part 3

    return (Max_dev,L1,L2,L3)


In [5]:
#  Sampling
def get_monte_carlo_points(N_interior, N_initial):
    # Sampler #1: PDE domain
    t_interior = np.random.uniform(low=T_min - 0.5*(T_max - T_min),
                           high=T_max,
                           size=[N_interior,1])
    s_interior = np.random.uniform(low=S_min - (S_max - S_min)*0.5,
                           high=S_max + (S_max - S_min)*0.5,
                           size=[N_interior,1])
    #you take all the t and the state space
    
    # Sampler #2: initial/terminal condition
    t_initial = T_max * np.ones((N_initial,1)) #Terminal condition
    s_initial = np.random.uniform(low=S_min - (S_max - S_min)*0.5,
                           high=S_max + (S_max - S_min)*0.5,
                           size=[N_initial,1])
    
    return (t_interior, s_interior, t_initial, s_initial)

In [6]:
def one_dimensional_bs_solution(time, state, strike_price, volatility, riskfree_rate):
    drift = riskfree_rate
    if np.size(volatility) == 1:  # scalar sigma
        volatility = volatility * np.ones(np.shape(time))
    if np.size(strike_price) == 1:  # scalar sigma
        strike_price = strike_price * np.ones(np.shape(time))
    if np.size(drift) == 1:  # scalar sigma
        drift = drift * np.ones(np.shape(time))
    if np.size(riskfree_rate) == 1:  # scalar sigma
        riskfree_rate = riskfree_rate * np.ones(np.shape(time))

    solution = np.zeros(np.shape(time))
    d1 = np.zeros(np.shape(time))
    d2 = np.zeros(np.shape(time))

    is_initial_time = (time == 0)
    is_zero = (state == 0)
    is_not_special_case = (time > 0) & (state > 0)

    d1[is_not_special_case] = 1. / (volatility[is_not_special_case] * np.sqrt(time[is_not_special_case])) * (
            np.log(state[is_not_special_case] / strike_price[is_not_special_case]) + (
            drift[is_not_special_case] + volatility[is_not_special_case] ** 2 * 0.5) * time[is_not_special_case])
    d2[is_not_special_case] = d1[is_not_special_case] \
                              - volatility[is_not_special_case] * np.sqrt(time[is_not_special_case])

    solution[is_not_special_case] = state[is_not_special_case] * norm.cdf(d1[is_not_special_case]) - strike_price[
        is_not_special_case] * np.exp(-riskfree_rate[is_not_special_case] * time[is_not_special_case]) * norm.cdf(
        d2[is_not_special_case])

    solution[is_initial_time] = np.maximum(0, state[is_initial_time] - strike_price[is_initial_time])
    solution[is_zero] = 0.
    return solution


In [7]:
nn_plot_list=[]
def compute_errors():
    max_error_container=[]
    mean_error_container=[]
    max_error=0
    times_to_evaluate_error=np.linspace(T_min,T_max,10)
    for t in times_to_evaluate_error:
        tt = t*np.ones_like(xplot.reshape(-1,1))
        nn_plot, = sess.run([vplot_t],
                        feed_dict={tplot_t:tt, xplot_t:xplot.reshape(-1,1)})
        nn_plot_list.append(nn_plot,)
        exact_plot = one_dimensional_bs_solution(
            T_max-tt, xplot.reshape(-1,1), strike_price, volatility, riskfree_rate)
        max_error=np.maximum(np.max(np.abs(nn_plot-exact_plot)),max_error) 
        mean_error_container.append(np.mean(np.abs(nn_plot-exact_plot)))
    New_mean_error_container=np.mean(mean_error_container)
    
    return max_error,New_mean_error_container

In [8]:
# Neural Network definition

model = DNN(nr_layers, nr_nodes_per_layer) #first time this model is constructed

t_interior_tf = tf.compat.v1.placeholder(tf.float32, shape=(None, 1), name="time_interior") #allows you to fill it with numbers
x_interior_tf = tf.compat.v1.placeholder(tf.float32, shape=(None, dimension_state), name="stock_prices_interior")
t_initial_tf = tf.compat.v1.placeholder(tf.float32, shape=(None, 1), name="time_initial")#allows you to fill it with numbers
x_initial_tf = tf.compat.v1.placeholder(tf.float32, shape=(None, dimension_state), name="stock_prices_initial")


Max_dev,residual_interior,residual_exterior,residual_initial= get_residual(model, t_interior_tf, x_interior_tf, t_initial_tf, x_initial_tf)
print("This is ")
print(residual_exterior)
print("This is ")
print(residual_interior)
residual = residual_interior + residual_initial+(residual_exterior*1) #this residual is the combination of the L1 and L2 norm
# Optimizer parameters
nr_steps = tf.Variable(0, trainable=False) #very weird itertation counter, counts the no of optimization steps we took
learning_rate = tf.compat.v1.train.exponential_decay(initial_learning_rate, nr_steps,
                                           learning_rate_decay_steps, 
                                           learning_rate_decay_rate, staircase=True)
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(residual) #defining the optimizer 
# gradient descent with a momemtum term. mizimised the residuals




# Plot tensors
tplot_t = tf.compat.v1.placeholder(tf.float32, [None,1], name="tplot_t") # We name to recover it later
xplot_t = tf.compat.v1.placeholder(tf.float32, [None,1], name="xplot_t")
vplot_t = tf.identity(model(tplot_t, xplot_t), name="vplot_t") # Trick for naming the trained model


# Training data holders
residuals_list_residual0_1=[]
residuals_list_residual1_1=[]
residuals_list_residual2_1=[]
residuals_list_residual3_1=[]
residuals_list_residual4_1=[]
residuals_list_MaxDev0_1=[]
residuals_list_MaxDev1_1=[]
residuals_list_MaxDev2_1=[]
residuals_list_MaxDev3_1=[]
residuals_list_MaxDev4_1=[]
residuals_list_L10_1=[]
residuals_list_L11_1=[]
residuals_list_L12_1=[]
residuals_list_L13_1=[]
residuals_list_L14_1=[]
residuals_list_L20_1=[]
residuals_list_L21_1=[]
residuals_list_L22_1=[]
residuals_list_L23_1=[]
residuals_list_L24_1=[]
residuals_list_L30_1=[]
residuals_list_L31_1=[]
residuals_list_L32_1=[]
residuals_list_L33_1=[]
residuals_list_L34_1=[]
# Train network!!
init_op = tf.compat.v1.global_variables_initializer()




Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
This is 
Tensor("Mean_1:0", shape=(), dtype=float32)
This is 
Tensor("Mean:0", shape=(), dtype=float32)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
# before opening a tensorflow session, close the old one if possible
n=0
while n<5:
    try:
        sess.close()
    except NameError:
        pass 
    sess =  tf.compat.v1.Session()
###################################################L2 NORM is multiplied by 1#######################################
    sess.run(init_op)

    print(n)
    if n+1==1:print("This is the first run of the neural network")
    elif n+1==2:print("This is the second run of the neural network")
    elif n+1==3:print("This is the third run of the neural network")
    elif n+1==4:print("This is the fourth run of the neural network")
    elif n+1==5:print("This is the fifth run of the neural network")
    for epoch in range(nr_epochs):
        t_interior_mc, x_interior_mc, t_initial_mc, x_initial_mc = get_monte_carlo_points(N_interior, N_initial)

        for _ in range(steps_per_sample):
             Max_deviation,residual_value, residual_interior_value,residual_exterior_value,residual_initial_value, _ = \
                 sess.run([Max_dev,residual, residual_interior, residual_exterior,residual_initial, optimizer],
                          feed_dict = {t_interior_tf:t_interior_mc, x_interior_tf:x_interior_mc,
                                        t_initial_tf:t_initial_mc, x_initial_tf:x_initial_mc})
            

        if n==0:residuals_list_residual0_1.append(residual_value),residuals_list_MaxDev0_1.append(Max_deviation),residuals_list_L10_1.append(residual_interior_value),residuals_list_L20_1.append(residual_exterior_value),residuals_list_L30_1.append(residual_initial_value)
        elif n==1:residuals_list_residual1_1.append(residual_value),residuals_list_MaxDev1_1.append(Max_deviation),residuals_list_L11_1.append(residual_interior_value),residuals_list_L21_1.append(residual_exterior_value),residuals_list_L31_1.append(residual_initial_value)
        elif n==2:residuals_list_residual2_1.append(residual_value),residuals_list_MaxDev2_1.append(Max_deviation),residuals_list_L12_1.append(residual_interior_value),residuals_list_L22_1.append(residual_exterior_value),residuals_list_L32_1.append(residual_initial_value)
        elif n==3:residuals_list_residual3_1.append(residual_value),residuals_list_MaxDev3_1.append(Max_deviation),residuals_list_L13_1.append(residual_interior_value),residuals_list_L23_1.append(residual_exterior_value),residuals_list_L33_1.append(residual_initial_value)
        elif n==4:residuals_list_residual4_1.append(residual_value),residuals_list_MaxDev4_1.append(Max_deviation),residuals_list_L14_1.append(residual_interior_value),residuals_list_L24_1.append(residual_exterior_value),residuals_list_L34_1.append(residual_initial_value)

        if (not np.mod(epoch, 100)) or epoch+1==nr_epochs:
            print("Stage: {:04d}, Loss: {:e}, Maximum_Deviation: {:e}, L1: {:e}, L2: {:e}, L3: {:e}".format(
                epoch,residual_value,Max_deviation, residual_interior_value,residual_exterior_value,residual_initial_value) )
    n=n+1

0
This is the first run of the neural network
Stage: 0000, Loss: 1.349727e+00, Maximum_Deviation: 1.362720e+00, L1: 4.633159e-02, L2: 5.375406e-01, L3: 7.658548e-01
Stage: 0100, Loss: 7.832768e-04, Maximum_Deviation: 2.417240e-02, L1: 1.975694e-04, L2: 4.371129e-05, L3: 5.419962e-04
Stage: 0200, Loss: 5.626092e-04, Maximum_Deviation: 4.252791e-02, L1: 7.245746e-05, L2: 3.476651e-05, L3: 4.553852e-04
Stage: 0300, Loss: 4.265458e-04, Maximum_Deviation: 4.451060e-02, L1: 5.872343e-05, L2: 3.543538e-05, L3: 3.323870e-04
Stage: 0400, Loss: 4.634018e-04, Maximum_Deviation: 4.177070e-02, L1: 6.492058e-05, L2: 3.488760e-05, L3: 3.635937e-04
Stage: 0500, Loss: 3.543728e-04, Maximum_Deviation: 4.157871e-02, L1: 4.441339e-05, L2: 2.794028e-05, L3: 2.820192e-04
Stage: 0600, Loss: 3.428472e-04, Maximum_Deviation: 3.397340e-02, L1: 3.888392e-05, L2: 2.429875e-05, L3: 2.796645e-04
Stage: 0700, Loss: 2.415922e-04, Maximum_Deviation: 2.280301e-02, L1: 3.513021e-05, L2: 2.571413e-05, L3: 1.807478e-04
St

Stage: 6900, Loss: 1.238703e-05, Maximum_Deviation: 1.571015e-03, L1: 4.108279e-06, L2: 1.421926e-08, L3: 8.264530e-06
Stage: 7000, Loss: 7.112174e-06, Maximum_Deviation: 1.370296e-03, L1: 2.343037e-06, L2: 3.536756e-09, L3: 4.765600e-06
Stage: 7100, Loss: 1.390249e-04, Maximum_Deviation: 1.634806e-02, L1: 2.654528e-05, L2: 9.406733e-06, L3: 1.030729e-04
Stage: 7200, Loss: 3.062675e-06, Maximum_Deviation: 4.170507e-03, L1: 8.478077e-07, L2: 5.799192e-08, L3: 2.156875e-06
Stage: 7300, Loss: 4.277876e-06, Maximum_Deviation: 3.894448e-03, L1: 1.713786e-06, L2: 1.070111e-07, L3: 2.457079e-06
Stage: 7400, Loss: 4.510183e-06, Maximum_Deviation: 4.338667e-03, L1: 1.697117e-06, L2: 7.978684e-08, L3: 2.733279e-06
Stage: 7500, Loss: 4.225855e-06, Maximum_Deviation: 3.761932e-03, L1: 1.494901e-06, L2: 6.264194e-08, L3: 2.668313e-06
Stage: 7600, Loss: 3.599525e-06, Maximum_Deviation: 2.115414e-03, L1: 1.892776e-06, L2: 3.202932e-08, L3: 1.674719e-06
Stage: 7700, Loss: 6.994186e-06, Maximum_Deviati

Stage: 5700, Loss: 8.290660e-05, Maximum_Deviation: 1.661539e-03, L1: 2.806636e-05, L2: 9.161382e-09, L3: 5.483108e-05
Stage: 5800, Loss: 2.082807e-05, Maximum_Deviation: 4.797339e-03, L1: 1.244383e-05, L2: 5.072164e-07, L3: 7.877019e-06
Stage: 5900, Loss: 9.780788e-06, Maximum_Deviation: 3.341794e-03, L1: 5.844894e-06, L2: 1.507891e-07, L3: 3.785105e-06
Stage: 6000, Loss: 1.055969e-04, Maximum_Deviation: 3.908753e-03, L1: 8.284059e-05, L2: 1.909493e-07, L3: 2.256535e-05
Stage: 6100, Loss: 7.990393e-06, Maximum_Deviation: 4.925251e-03, L1: 2.387787e-06, L2: 1.893285e-07, L3: 5.413278e-06
Stage: 6200, Loss: 8.738509e-06, Maximum_Deviation: 3.999829e-03, L1: 3.329023e-06, L2: 1.491857e-07, L3: 5.260300e-06
Stage: 6300, Loss: 1.043789e-05, Maximum_Deviation: 2.074361e-03, L1: 2.777711e-06, L2: 1.996794e-08, L3: 7.640215e-06
Stage: 6400, Loss: 8.086488e-06, Maximum_Deviation: 4.693508e-03, L1: 1.689955e-06, L2: 5.022221e-07, L3: 5.894311e-06
Stage: 6500, Loss: 4.478867e-05, Maximum_Deviati

Stage: 4500, Loss: 2.847106e-05, Maximum_Deviation: 6.229520e-03, L1: 6.053979e-06, L2: 9.427300e-07, L3: 2.147436e-05
Stage: 4600, Loss: 6.681007e-05, Maximum_Deviation: 2.808571e-03, L1: 2.128779e-05, L2: 1.139224e-07, L3: 4.540836e-05
Stage: 4700, Loss: 1.597720e-05, Maximum_Deviation: 7.127643e-03, L1: 3.884858e-06, L2: 8.703580e-07, L3: 1.122198e-05
Stage: 4800, Loss: 4.027418e-05, Maximum_Deviation: 7.416606e-03, L1: 5.459971e-06, L2: 1.654229e-06, L3: 3.315998e-05
Stage: 4900, Loss: 2.289127e-04, Maximum_Deviation: 1.768446e-02, L1: 6.288679e-05, L2: 2.358484e-05, L3: 1.424411e-04
Stage: 5000, Loss: 1.484112e-05, Maximum_Deviation: 5.581021e-03, L1: 3.546233e-06, L2: 5.031354e-07, L3: 1.079176e-05
Stage: 5100, Loss: 2.774787e-05, Maximum_Deviation: 4.356503e-03, L1: 4.135954e-06, L2: 2.899954e-07, L3: 2.332192e-05
Stage: 5200, Loss: 2.289646e-05, Maximum_Deviation: 7.174611e-03, L1: 6.839091e-06, L2: 9.151778e-07, L3: 1.514219e-05
Stage: 5300, Loss: 4.093346e-05, Maximum_Deviati

Stage: 3300, Loss: 1.578425e-05, Maximum_Deviation: 5.276024e-03, L1: 3.488352e-06, L2: 8.738880e-07, L3: 1.142201e-05
Stage: 3400, Loss: 3.290618e-05, Maximum_Deviation: 2.263367e-03, L1: 8.204531e-06, L2: 1.143410e-08, L3: 2.469021e-05
Stage: 3500, Loss: 2.635144e-05, Maximum_Deviation: 2.069831e-03, L1: 9.065566e-06, L2: 1.211858e-08, L3: 1.727375e-05
Stage: 3600, Loss: 8.019225e-05, Maximum_Deviation: 6.522000e-03, L1: 2.335098e-05, L2: 2.676349e-07, L3: 5.657364e-05
Stage: 3700, Loss: 4.183829e-05, Maximum_Deviation: 6.320834e-03, L1: 1.573267e-05, L2: 1.370707e-06, L3: 2.473491e-05
Stage: 3800, Loss: 1.739927e-05, Maximum_Deviation: 3.344417e-03, L1: 5.637025e-06, L2: 6.231563e-08, L3: 1.169993e-05
Stage: 3900, Loss: 9.703922e-06, Maximum_Deviation: 4.715383e-03, L1: 2.849926e-06, L2: 4.385114e-07, L3: 6.415485e-06
Stage: 4000, Loss: 1.858484e-05, Maximum_Deviation: 6.306708e-03, L1: 5.749803e-06, L2: 8.060929e-07, L3: 1.202895e-05
Stage: 4100, Loss: 1.133530e-05, Maximum_Deviati

Stage: 2100, Loss: 6.473569e-05, Maximum_Deviation: 9.691060e-03, L1: 9.829463e-06, L2: 1.902590e-06, L3: 5.300363e-05
Stage: 2200, Loss: 5.210121e-05, Maximum_Deviation: 1.109827e-02, L1: 9.820498e-06, L2: 4.552051e-06, L3: 3.772866e-05
Stage: 2300, Loss: 7.452020e-05, Maximum_Deviation: 9.430766e-03, L1: 1.306099e-05, L2: 3.107676e-06, L3: 5.835153e-05
Stage: 2400, Loss: 3.322871e-05, Maximum_Deviation: 7.928640e-03, L1: 8.785191e-06, L2: 1.953735e-06, L3: 2.248979e-05
Stage: 2500, Loss: 4.418209e-05, Maximum_Deviation: 6.710052e-03, L1: 6.643264e-06, L2: 1.135305e-06, L3: 3.640352e-05
Stage: 2600, Loss: 3.600227e-05, Maximum_Deviation: 8.470297e-03, L1: 5.861550e-06, L2: 2.152028e-06, L3: 2.798869e-05
Stage: 2700, Loss: 3.697769e-05, Maximum_Deviation: 9.095758e-03, L1: 8.813716e-06, L2: 3.519276e-06, L3: 2.464469e-05
Stage: 2800, Loss: 2.488460e-05, Maximum_Deviation: 7.534057e-03, L1: 7.077152e-06, L2: 1.822363e-06, L3: 1.598508e-05
Stage: 2900, Loss: 4.613861e-05, Maximum_Deviati

In [11]:
# Plot results
N = 41      # Points on plot grid

times_to_plot = [0*T_max, 0.33*T_max, 0.66*T_max, T_max]
tplot = np.linspace(T_min, T_max, N) # for surface plot
xplot = np.linspace(S_min, S_max, N)

In [12]:
max_error_1,New_mean_error_container_1=compute_errors()
print(max_error_1)
print(New_mean_error_container_1)
print(nn_plot_list)

0.015301346728869625
0.0014952019578190444
[array([[0.00223151],
       [0.00209045],
       [0.0019823 ],
       [0.00190312],
       [0.00184086],
       [0.00177777],
       [0.00169611],
       [0.00158691],
       [0.00146252],
       [0.00136882],
       [0.00139913],
       [0.0017069 ],
       [0.00251171],
       [0.00409898],
       [0.00680572],
       [0.01099032],
       [0.01698852],
       [0.02506277],
       [0.03536007],
       [0.04789311],
       [0.0625506 ],
       [0.07913125],
       [0.09738508],
       [0.11705181],
       [0.13788468],
       [0.15966311],
       [0.18219736],
       [0.20532754],
       [0.22892138],
       [0.25287095],
       [0.27708986],
       [0.30151078],
       [0.3260819 ],
       [0.35076544],
       [0.37553397],
       [0.40036902],
       [0.42525896],
       [0.4501967 ],
       [0.47517982],
       [0.50020766],
       [0.5252814 ]], dtype=float32), array([[0.00225192],
       [0.0020934 ],
       [0.00197592],
       [0.00190